In [26]:
import rasterio
import fiona
from rasterio.features import rasterize
from rasterio.transform import from_origin
from collections import defaultdict
from shapely.geometry import Point
import numpy as np
import os
from pathlib import Path


# Define file paths
# input_gpkg = '../downloads/MSG_2ND/kisumu/KISUMU_LST_2024-04-01.gpkg'  # Path to .gpkg file
# output_template = '../lst_rasters/kisumu/KISUMU_LST_2024-04-01_{:02d}.tiff'  # Output file template with hour placeholder

# Define file path
input_gpkg = Path('../downloads/MSG_2ND/roma/LST_2024-06-01.gpkg')

# Extract basefolder (city name)
city = input_gpkg.parent.name
print(city)  # Output: roma

# Extract layername from filename
layername = input_gpkg.stem
print(layername)  # Output: LST_2024-01-02

# Construct output template with hour placeholder
output_template = f'../lst_rasters/{city}/MSG_{layername}_{{:02d}}.tiff'
print(output_template)

# Open GeoPackage file with Fiona
with fiona.open(input_gpkg, layer=layername) as src:

    # Print fields for debugging
    print("Fields in GeoPackage:", src.schema['properties'])

    # Extract geometries and LST values, grouped by hour
    data_by_hour = defaultdict(list)
    all_points = []

    for feature in src:
        x, y = feature['geometry']['coordinates']  # Extract coordinates
        geom = Point(x, y)
        hour = feature['properties']['hour']
        temperature = feature['properties']['temperature']

        # Treat None (NA) temperatures as -9999
        temperature = -9999 if temperature is None else float(temperature)  # Ensure float32 format

        # Store data for each hour
        data_by_hour[hour].append((geom, temperature))
        all_points.append((x, y))

    # Convert to NumPy array for distance calculations
    all_points = np.array(all_points)

    # Compute pixel size using adjacent geopoints
    def compute_min_distance(points, axis=0):
        """ Compute the minimum distance between adjacent points along the specified axis (0=X, 1=Y) """
        sorted_points = np.sort(points[:, axis])  # Sort by X or Y
        diffs = np.diff(sorted_points)  # Compute differences
        valid_diffs = diffs[diffs > 0]  # Filter out zero distances
        return np.min(valid_diffs) if valid_diffs.size > 0 else 0.01  # Default fallback

    pixel_size_x = compute_min_distance(all_points, axis=0)  # Horizontal resolution
    pixel_size_y = compute_min_distance(all_points, axis=1)  # Vertical resolution

    # Get bounding box of all data points
    min_x, min_y = np.min(all_points[:, 0]), np.min(all_points[:, 1])
    max_x, max_y = np.max(all_points[:, 0]), np.max(all_points[:, 1])

    # Compute raster dimensions
    width = int(np.round((max_x - min_x) / pixel_size_x)) + 1
    height = int(np.round((max_y - min_y) / pixel_size_y)) + 1

    # Adjust transform to avoid shifting the grid
    transform = from_origin(min_x - (pixel_size_x / 2), max_y + (pixel_size_y / 2), pixel_size_x, pixel_size_y)

    # Ensure all 24 hours are processed
    all_hours = range(24)

    for hour in all_hours:
        output_tif = output_template.format(hour)  # Format filename with 2-digit hour
        shapes = data_by_hour.get(hour, [])  # Get data for the hour

        # If data exists, rasterize points with computed pixel size
        if shapes:
            points = [(geom, temp) for geom, temp in shapes]

            burned = rasterize(
                points,
                out_shape=(height, width),
                transform=transform,
                fill=-9999,
                dtype='float32',
                all_touched=True  # Ensures more pixel coverage
            )
        else:
            burned = np.full((height, width), -9999, dtype=np.float32)  # Empty raster

        # **Only save the raster if there is at least one non-No-Data pixel**
        if np.any(burned != -9999):  # Check if there is at least one valid pixel
            with rasterio.open(output_tif, 'w', driver='GTiff', height=height, width=width,
                               count=1, dtype='float32', crs=src.crs, transform=transform,
                               nodata=-9999) as dst:
                dst.write(burned, 1)  # Single-band raster

            print(f"✅ Saved: {output_tif} (At least one valid pixel)")
        else:
            print(f"❌ Skipped: {output_tif} (All pixels are No-Data)")

print("Rasterization complete!")


roma
LST_2024-06-01
../lst_rasters/roma/MSG_LST_2024-06-01_{:02d}.tiff
Fields in GeoPackage: {'time': 'datetime', 'temperature': 'float', 'hour': 'int32', 'day': 'int32', 'month': 'int32', 'year': 'int32'}
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_00.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_01.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_02.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_03.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_04.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_05.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_06.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_07.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01_08.tiff (At least one valid pixel)
✅ Saved: ../lst_rasters/roma/MSG_LST_2024-06-01